# Overall Statistics

This notebook contains overall statistics of the `PA-2016` (Pennsylvania, year 2016) dataset and should be Part 1 of our presentation.

## Neccesary Pre-processing Goes Here

Cells are copied from the main notebook.

In [1]:
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Display otpions
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 100)
np.set_printoptions(linewidth=100)

In [3]:
# 11 primary (more important) tables
# Note that one incident is associated with up to 10 offenses
primaryTables = [
    "nibrs_arrestee", "nibrs_bias_motivation", "nibrs_incident", "nibrs_month",
    "nibrs_offender", "nibrs_offense", "nibrs_property_desc", "nibrs_property",
    "nibrs_victim_injury", "nibrs_victim_offender_rel", "nibrs_victim"
]

# 23 lookup tables as a dictionary of {table_name: (oldCol, newCol)} format
lookupTables = {
    "nibrs_activity_type": None, "nibrs_age": None, "nibrs_arrest_type": None, 
    "nibrs_assignment_type": None, "nibrs_bias_list": None, "nibrs_circumstances": None,
    "nibrs_cleared_except": None,
    "nibrs_criminal_act_type": ("criminal_act_id", "criminal_act_name"),
    "nibrs_drug_measure_type": None, "nibrs_ethnicity": None,
    "nibrs_injury": None, "nibrs_justifiable_force": None,
    "nibrs_location_type": ("location_id", "location_name"),
    "nibrs_offense_type" : ("offense_type_id", "offense_name"), "nibrs_prop_desc_type": None,
    "nibrs_prop_loss_type": None,
    "nibrs_relationship": ("relationship_id", "relationship_name"),
    "nibrs_suspected_drug_type": None, "nibrs_using_list": None,
    "nibrs_victim_type": ("victim_type_id", "victim_type_name"),
    "nibrs_weapon_type": None, "ref_race": None, "ref_state": None
}

# 9 secondary (less important, just for human differentiation) tables
secondaryTables = [
    "agency_participation", "cde_agencies", "nibrs_arrestee_weapon", "nibrs_criminal_act",
    "nibrs_suspected_drug", "nibrs_suspect_using", "nibrs_victim_circumstances",
    "nibrs_victim_offense", "nibrs_weapon"
]

In [4]:
def readCSVs(dataset):
    """
    Read DataFrames into a dictionary with table names as keys
    Args:
        dataset (string): NIBRS dataset name, e.g., SC-2016
    Return:
        dic (dictionary): dictionary of DataFrames
    """
    print('Reading all CSVs for %s dataset...' % dataset)
    dic = {}
    for table in primaryTables + list(lookupTables.keys()) + secondaryTables:
        dic[table] = pd.read_csv("%s/%s.csv" % (dataset, table))
    print('Finished reading %d CSVs' % len(dic))
    return dic

PA2016 = readCSVs('PA-2016')
# SC2016 = readCSVs('SC-2016')

Reading all CSVs for PA-2016 dataset...
Finished reading 43 CSVs


In [5]:
# Replace a col with a specified mapping dict:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html
# https://stackoverflow.com/questions/20250771/remap-values-in-pandas-column-with-a-dict
def createMappings(dfs):
    """
    Create a nested dictionary of mappings 
    Args:
        dfs (dictionary): dictionary of DataFrames previously read
    Return:
        mappings (nested dictionary): {(oldCol, newCol): {oldValue: newValue}}
    """
    print('Creating lookup table mappings...')
    mappings = {}
    for table in lookupTables:
        # Only handle those with defined mapping
        if lookupTables[table] is not None:
            oldCol, newCol = lookupTables[table]
            mappings[(oldCol, newCol)] = dict(zip(dfs[table][oldCol], dfs[table][newCol]))
    print('Finished creating mappings from %d lookup tables' % len(mappings))
    return mappings

maps = createMappings(PA2016)

Creating lookup table mappings...
Finished creating mappings from 5 lookup tables


In [6]:
def replaceAndRename(df, lookupTableNames):
    """
    Replace and rename DataFrame columns in place.
    Args:
        df (DataFrame): DataFrame to be replaced
        lookupTableNames (list): List of lookup table names for replacing and renaming
    """
    # Call df.replace() and df.rename() once for each changing column
    for lookupTableName in lookupTableNames:
        # Build the nested dictionary required by df.replace()
        oldCol, newCol = lookupTables[lookupTableName]
        dic = {oldCol: maps[(oldCol, newCol)]}        
        df.replace(dic, inplace=True)
        df.rename(columns={oldCol: newCol}, inplace=True)

print('Replacing and Renaming columns...')
replaceAndRename(PA2016['nibrs_offense'], ['nibrs_location_type', 'nibrs_offense_type'])
replaceAndRename(PA2016['nibrs_victim'], ['nibrs_victim_type'])
replaceAndRename(PA2016['nibrs_victim_offender_rel'], ['nibrs_relationship'])
print('Done')
# SC2016['nibrs_offense'].head()
# SC2016['nibrs_victim'].head()

Replacing and Renaming columns...
Done


In [7]:
# Helper function for plotting sns.countplot given df, col, and topK
def countPlot(df, col, topK=None, dpi = 100):
    plt.figure(figsize=(8,6), dpi=dpi)
    # Only show top K counts. Default: all counts
    sns.countplot(y = df[col], order = df[col].value_counts().index[:topK])

## Exploratory Data Analysis: Overall Statistics

Due to the schema design of the incident and offense DataFrames, it is more convenient to 
merge them from the beginning. Let's name it as `incident_offense`.

In [8]:
# Temp
PA2016.keys()

dict_keys(['nibrs_arrestee', 'nibrs_bias_motivation', 'nibrs_incident', 'nibrs_month', 'nibrs_offender', 'nibrs_offense', 'nibrs_property_desc', 'nibrs_property', 'nibrs_victim_injury', 'nibrs_victim_offender_rel', 'nibrs_victim', 'nibrs_activity_type', 'nibrs_age', 'nibrs_arrest_type', 'nibrs_assignment_type', 'nibrs_bias_list', 'nibrs_circumstances', 'nibrs_cleared_except', 'nibrs_criminal_act_type', 'nibrs_drug_measure_type', 'nibrs_ethnicity', 'nibrs_injury', 'nibrs_justifiable_force', 'nibrs_location_type', 'nibrs_offense_type', 'nibrs_prop_desc_type', 'nibrs_prop_loss_type', 'nibrs_relationship', 'nibrs_suspected_drug_type', 'nibrs_using_list', 'nibrs_victim_type', 'nibrs_weapon_type', 'ref_race', 'ref_state', 'agency_participation', 'cde_agencies', 'nibrs_arrestee_weapon', 'nibrs_criminal_act', 'nibrs_suspected_drug', 'nibrs_suspect_using', 'nibrs_victim_circumstances', 'nibrs_victim_offense', 'nibrs_weapon'])

In [9]:
# PA2016['incident_offense'] = PA2016['nibrs_incident'].merge()
nIncidents = len(PA2016['nibrs_incident'])
nOffenses = len(PA2016['nibrs_offense'])
print('There are a total of %d inccidents, %d offenses (%.2f offenses per incident)' %
      (nIncidents, nOffenses, nOffenses / nIncidents))


There are a total of 5235 inccidents, 6107 offenses (1.17 offenses per incident)


In [10]:
# Join incident and offense DFs

### Merge Incident and Offense DF

In [11]:
incidentOffenseJoinDF = incidentDF.merge(offenseDF, on='incident_id')
# Indeed, all offenses map to one incident
print('len(incidentOffenseJoinDF): %d' % len(incidentOffenseJoinDF))

NameError: name 'incidentDF' is not defined

In [ ]:
incidentOffenseJoinDF.head()

In [ ]:
incidentOffenseJoinDF.replace({"nibrs_month_id": monthMapping, "offense_type_id": offenseTypeMapping}, inplace=True)

In [ ]:
# Offense distribution by month
incidentOffenseJoinDF[["nibrs_month_id", "offense_id"]].groupby("nibrs_month_id").count()

In [ ]:
# Offense distribution by (offense type, month)
offenseTypeByMonth = incidentOffenseJoinDF\
    .groupby(["offense_type_id", "nibrs_month_id"])["offense_id"].count()\
    .unstack(fill_value=0).as_matrix()
offenseTypeByMonth

In [ ]:
peakMonth = np.argmax(offenseTypeByMonth, axis=1) + 1
offMonth = np.argmin(offenseTypeByMonth, axis=1) + 1
print("<Offense Type, Peak Month, Off Month>")
list(zip(sorted(incidentOffenseJoinDF["offense_type_id"].unique()), peakMonth, offMonth))